# Beach Sediment Classification

Notebook environment to migrate netcdf files to CF compliant zarr

In [101]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

### Configure OS independent paths

In [102]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))
sys.path.append(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\coclicodata')

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = p_drive.joinpath("1000545-054-globalbeaches", "15_GlobalCoastalAtlas", "datasets")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

<IPython.core.display.Javascript object>

In [103]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath("04_Beachsediment_class")
dataset_dir_beachclass = dataset_dir.joinpath("beachsediment_class.nc")
dataset_out_file = "BeachSediment_Class"
CF_dir = gca_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

In [104]:
# # write csv to netcdf

# # Load data from CSV file into a pandas dataframe
# csv_dir = r'P:\1000545-054-globalbeaches\07_Muddy_Coasts\Paper Final Scripts and Data\Data\WorldClassifiedTransects2020.csv'
# df = pd.read_csv(csv_dir)

<IPython.core.display.Javascript object>

In [105]:
# # Convert the pandas dataframe to an xarray dataset
# ds = xr.Dataset.from_dataframe(df)

# # Write the xarray dataset to a netCDF file
# ds.to_netcdf(dataset_dir_beachclass)

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [106]:
# open datasets
ds_sed = xr.open_dataset(dataset_dir_beachclass)

# check original dataset
ds_sed

<xarray.Dataset>
Dimensions:        (index: 1826995)
Coordinates:
  * index          (index) float64 nan nan nan nan nan ... nan nan nan nan nan
Data variables: (12/34)
    Unnamed: 0.2   (index) int64 ...
    Unnamed: 0     (index) int64 ...
    transect_id    (index) object ...
    country_name   (index) object ...
    continent      (index) object ...
    flag_sandy     (index) bool ...
    ...             ...
    mintemp        (index) float64 ...
    mhhw           (index) float64 ...
    mllw           (index) float64 ...
    tidal range    (index) float64 ...
    prediction     (index) float64 ...
    Unnamed: 0.1   (index) float64 ...

<IPython.core.display.Javascript object>

In [107]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=dataset_dir_beachclass, 
                 working_dir=CF_dir
                 )


<IPython.core.display.Javascript object>

In [108]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_dir_beachclass, working_dir=CF_dir)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\04_Beachsediment_class\\beachsediment_class.nc': {'warnings': '48', 'errors': '5'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [109]:
import json

# NetCDF attribute alterations
f_global = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\04_Beachsediment_class\metadata_beachsediment.json')
meta_global = json.load(f_global)

for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    ds_sed.attrs[attr_name] = attr_val

ds_sed.attrs['Conventions'] = "CF-1.8"

<IPython.core.display.Javascript object>

In [110]:
ds_sed

<xarray.Dataset>
Dimensions:        (index: 1826995)
Coordinates:
  * index          (index) float64 nan nan nan nan nan ... nan nan nan nan nan
Data variables: (12/34)
    Unnamed: 0.2   (index) int64 ...
    Unnamed: 0     (index) int64 ...
    transect_id    (index) object ...
    country_name   (index) object ...
    continent      (index) object ...
    flag_sandy     (index) bool ...
    ...             ...
    mintemp        (index) float64 ...
    mhhw           (index) float64 ...
    mllw           (index) float64 ...
    tidal range    (index) float64 ...
    prediction     (index) float64 ...
    Unnamed: 0.1   (index) float64 ...
Attributes: (12/21)
    TITLE:               Beach Sediment Classification
    TITLE_ABBREVIATION:  sed_class
    DESCRIPTION:         The Beach Sediment Classification dataset indicates ...
    SHORT_DESCRIPTION:   The Beach Sediment Classification dataset indicates ...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 
    LONG_NAME:           BEACH_SEDIMENT_CLASSIFICATION
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [111]:
# combine start and end coordinates into a transect
from shapely.geometry import LineString

start_lons = ds_sed["New_Start_lon"].values
start_lats = ds_sed["New_Start_lat"].values
end_lons = ds_sed["New_End_lon"].values
end_lats = ds_sed["New_End_lat"].values
coords = zip(zip(start_lons, start_lats), zip(end_lons, end_lats))

ds_sed["geometry"] = (
    ["index"],
    [str(LineString(line)) for line in coords],
)
ds_sed["geometry"].attrs["long_name"] = "Geometry"

<IPython.core.display.Javascript object>

In [112]:
keep_vars = ['transect_id', 'country_name', 'continent', 'New_Start_lon', 'New_Start_lat', 'prediction', 'geometry']
allvars = list(ds_sed.keys())
delete_vars = list(set(allvars).difference(set(keep_vars)))

ds_sed = ds_sed.drop(delete_vars)

<IPython.core.display.Javascript object>

In [113]:
ds_sed

<xarray.Dataset>
Dimensions:        (index: 1826995)
Coordinates:
  * index          (index) float64 nan nan nan nan nan ... nan nan nan nan nan
Data variables:
    transect_id    (index) object ...
    country_name   (index) object ...
    continent      (index) object ...
    New_Start_lon  (index) float64 -72.98 -72.99 -72.99 ... 39.89 39.88 39.88
    New_Start_lat  (index) float64 -54.46 -54.46 -54.46 ... 65.62 65.62 65.62
    prediction     (index) float64 ...
    geometry       (index) <U93 'LINESTRING (-72.98252423386046 -54.459553896...
Attributes: (12/21)
    TITLE:               Beach Sediment Classification
    TITLE_ABBREVIATION:  sed_class
    DESCRIPTION:         The Beach Sediment Classification dataset indicates ...
    SHORT_DESCRIPTION:   The Beach Sediment Classification dataset indicates ...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 
    LONG_NAME:           BEACH_SEDIMENT_CLASSIFICATION
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [114]:
def replace_labels(x):
    label_dict = {0: 'sand', 1 : 'mud', 2: 'coastal_cliff', 3: 'vegetated', 4: 'other'}
    if not pd.isna(x):
        return label_dict[x]
    else:
        return 'other'

prediction_labels = xr.apply_ufunc(replace_labels, 
                                    ds_sed['prediction'], 
                                    vectorize= True)

ds_sed = ds_sed.drop('prediction')
ds_sed["prediction"] = ("index", prediction_labels.data)

<IPython.core.display.Javascript object>

In [115]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
ds_sed = ds_sed.rename_dims({"index": "stations"})

f_vars = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\04_Beachsediment_class\vars_beachsediment.json')
meta_vars = json.load(f_vars)

for var_name, var_dict in meta_vars.items():
    ds_sed = ds_sed.rename_vars({var_name : var_dict['name']})
    for key, value in var_dict.items():
        if key != 'name':
            ds_sed[var_dict['name']].attrs[key] = value


# change dtypes
object_vars = ['transect_id', 'country', 'continent', 'geometry', 'sediment_label']
for ov in object_vars:
    ds_sed[ov] = ds_sed[ov].astype('S')

coords = [x for x in list(ds_sed.keys()) if x != 'sediment_label']
ds_sed = ds_sed.set_coords(coords)

# drop index xarray
ds_sed = ds_sed.drop('index')

<IPython.core.display.Javascript object>

In [116]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first
# dataset["scenarios"]
ds_sed

<xarray.Dataset>
Dimensions:         (stations: 1826995)
Coordinates:
    transect_id     (stations) |S15 b'BOX_028_000_0' ... b'BOX_211_092_29'
    country         (stations) |S40 b'Chile' b'Chile' ... b'Russia' b'Russia'
    continent       (stations) |S23 b'South America' ... b'Europe'
    lon             (stations) float64 -72.98 -72.99 -72.99 ... 39.88 39.88
    lat             (stations) float64 -54.46 -54.46 -54.46 ... 65.62 65.62
    geometry        (stations) |S93 b'LINESTRING (-72.98252423386046 -54.4595...
Dimensions without coordinates: stations
Data variables:
    sediment_label  (stations) |S13 b'coastal_cliff' ... b'sand'
Attributes: (12/21)
    TITLE:               Beach Sediment Classification
    TITLE_ABBREVIATION:  sed_class
    DESCRIPTION:         The Beach Sediment Classification dataset indicates ...
    SHORT_DESCRIPTION:   The Beach Sediment Classification dataset indicates ...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 
    LONG_NAME:           BEACH_SEDIMENT_CLASSIFICATION
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [117]:
# save new .nc files
ds_sed.to_netcdf(path=str(dataset_dir_beachclass).replace(".nc", "_CF.nc"))

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [118]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(dataset_dir_beachclass).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [119]:
# save altered CF compliancy
save_compliancy(
    cap,
    testfile=str(dataset_dir_beachclass).replace(".nc", "_CF.nc"),
    working_dir=CF_dir,
)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\04_Beachsediment_class\\beachsediment_class_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

### write data to Zarr files

In [120]:
# export to zarr in write mode (to overwrite if exists)
ds_sed.to_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file), mode="w")

<IPython.core.display.Javascript object>

In [121]:
test = xr.open_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file))

<IPython.core.display.Javascript object>